In [ ]:
from os import environ
from tempfile import mkdtemp

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', mkdtemp())

TOOL_ID = 'ONSSET'
output_folder

In [ ]:
import pandas as pd
from os.path import join, splitext

specifications_path = join(output_folder, 'specifications.xlsx')
json_output_path = join(output_folder, 'assets.json')

excel_writer = pd.ExcelWriter(specifications_path)
for file_name in [
    'specs_data.csv',
    'scenario_info.csv',
    'scenario_parameters.csv',
]:
    table = pd.read_csv(join(input_folder, file_name))
    base_name = splitext(file_name)[0]
    sheet_name = ''.join(_.title() for _ in base_name.split('_'))
    table.to_excel(excel_writer, sheet_name=sheet_name, index=False)
excel_writer.save()

In [ ]:
from os import makedirs
from os.path import join

details_folder = join(output_folder, 'details')
summaries_folder = join(output_folder, 'summaries')
try:
    makedirs(details_folder)
    makedirs(summaries_folder)
except OSError:
    pass

In [ ]:
from onsset.runner import calibration, scenario

cities_path = join(input_folder, 'cities.csv')
calibrated_specifications_path = join(output_folder, 'specifications.xlsx')
calibrated_cities_path = join(output_folder, 'cities.csv') 

calibration(
    specs_path=specifications_path,
    csv_path=cities_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path,
    results_folder=details_folder,
    summary_folder=summaries_folder)

In [ ]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    pd.concat(tables).to_csv(target_path, index=False)
    
collect_scenarios(join(output_folder, 'details.csv'), details_folder)
collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)

In [ ]:
calculated_data_path = join(output_folder, 'details.csv')

base_electricity_structure = {
    'assets': {
        'assetById': {},
        'assetTypeByCode': {},
        'assetsGeoJson': {},
        'styles': {}
    }
}

table = pd.read_csv(calculated_data_path)

summary = table.describe()

reserved_cols = {'X_deg','Y_deg', 'Country', 'id', 'Elevation', 'ScenarioName', }

columns = [*set(table.columns) - reserved_cols]
table['id'] = table['ScenarioName'] + '_' + table['id'].astype(str)
table.set_index('id')

columns

In [ ]:
def quantile_strategie(column, row, summary):
    try:
        if row[column] == summary[column]['min']:
            return 'min'
        elif row[column] <= summary[column]['25%']:
            return '25%'
        elif row[column] <= summary[column]['50%']:
            return '50%'
        elif row[column] <= summary[column]['75%']:
            return '75%'
        else:
            return 'max'
    except KeyError:
        return row[column]

def get_name(row):
    return f'{row["Country"]} {row["id"].split("_")[-1]}'

def generate_assets_by_id(df):
    assets_by_id = {}
    for index, row in df.iterrows():
        assets_by_id[row['id']] = {
            "name": get_name(row),
            "typeCode": "m",
            "utilityId": "",
            "connections": {},
            'attributes': row[[*columns]].to_dict(),
        }
        
    return assets_by_id

def generate_assets_geojson(df):
    features = [{
        "geometry": {
            "type": "Point",
            "coordinates": [
              row['X_deg'],
              row['Y_deg'],
            ]
        },
        "type": "Feature",
        "properties": {
            "id": row['id'],
            "typeCode": "m",
            'styles': {
                TOOL_ID: {column: quantile_strategie(column, row, summary) for column in columns}
            }
        }
    } for index, row in df.iterrows()]
    
    return {
      "type": "FeatureCollection",
      "features": features
    }

assets_geojson = generate_assets_geojson(table.head(100))
assets_by_id =  generate_assets_by_id(table.head(100))


#summary['Minimum_Tech_Off_grid2030']
#assets_by_id

In [ ]:
gray_color = [128, 128, 128, 160]
blue_color = [3, 169, 244, 160]
green_color = [92, 176, 81, 160]
yellow_color = [255, 255, 0, 160]
red_color = [255, 0, 0, 160]

inner_style_dictionary = {
    'min': {
        'fillColor': gray_color,
        'lineColor': gray_color,
        'lineWidth': 5
        # 'dashArray': [1, 1]
    },
    '25%': {
        'fillColor': blue_color,
        'lineColor': blue_color,
        'lineWidth': 10
        # 'dashArray': [1, 1]
    },
    '50%': {
        'fillColor': green_color,
        'lineColor': green_color,
        'lineWidth': 15
        # 'dashArray': [1, 1]
    },
    '75%': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
    'max': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
}


categorical_columns = list(table.select_dtypes(include=['object']).columns)
general_styles = { column: inner_style_dictionary for column in columns if column not in categorical_columns }
general_styles['CommercialDemand']


In [ ]:
def get_type(column):
    if column not in categorical_columns:
        return column, 'number'
    
    return column, 'string'
    
asset_types = {
    'm': {
        'assetAttributes': [
          get_type(column)  for column in columns
        ]
    }
}


asset_types

In [ ]:
import json

base_electricity_structure['assets']['assetTypeByCode'] = asset_types
base_electricity_structure['assets']['assetsGeoJson'] = assets_geojson
base_electricity_structure['assets']['assetById'] = assets_by_id
base_electricity_structure['assets']['styles'][TOOL_ID] = general_styles

json.dump(base_electricity_structure, open(json_output_path, 'wt'), indent=4)